In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

# folder_name = input('folder name\t')

# project_path = f"/content/drive/MyDrive/{folder_name}"
project_path = f'/content/drive/MyDrive/meli_challenge_202404-main'

# crear folder donde guardaremos todo lo del proyecto
os.chdir(project_path)


In [ ]:
!pwd
!ls

In [ ]:
import pandas as pd


# tomar los valores pre cargados
try:
    df_seller_skus = pd.read_csv('preload_seller_skus.csv')
    df_category_fee = pd.read_csv('preload_category_fee.csv')
except ValueError:
    print('error')

In [ ]:
df_con_fee = df_seller_skus.merge(df_category_fee, how='left', on=['category_id','listing_type_id']).copy()

df_con_fee

In [ ]:
def fee_per_unit(price, fee):

    if price >= 299:

        return round(price*fee,6)

    elif 149 <= price < 299:

        return round(price*fee + 30,6)

    elif price < 149:

        return round(price*fee + 25,6)


In [ ]:
# df_con_fee['fee_per_unit'] = fee_per_unit(df_con_fee['price'], df_con_fee['listing_type_fee'])

# df_con_fee['fee_per_unit'] = df_con_fee[['price', 'listing_type_fee']].apply(fee_per_unit)
df_con_fee['fee_per_unit'] = df_con_fee.apply(lambda row: fee_per_unit(row['price'], row['listing_type_fee']), axis=1)
df_con_fee['stock_value'] = round(df_con_fee['stock'] * df_con_fee['price'],2)
df_con_fee['stock_tgmv'] =  round(df_con_fee['stock'] * df_con_fee['fee_per_unit'],2)

In [ ]:
df_con_fee

In [ ]:
sellet_logistic = df_con_fee.groupby(by=['seller_nickname', 'logistic_type'])['stock'].sum().reset_index().copy()

sellet_logistic['stock_share'] = round(sellet_logistic['stock'] / sellet_logistic.groupby('seller_nickname')['stock'].transform('sum'),3)
# df_con_fee.groupby(by=['seller_nickname', 'logistic_type'])['stock'].transform('sum')

In [ ]:
seller_full = list(sellet_logistic[(sellet_logistic['logistic_type'] == 'fulfillment') & (sellet_logistic['stock_share'] >= 0.5)]['seller_nickname'].unique())

seller_full

In [ ]:
len(seller_full)

In [ ]:
def definir_rotacion(stock, logistic_type):

    if logistic_type == 'fulfillment':
        value = stock/3

    else:
        value = stock/6

    return round(value,2)

In [ ]:
# seller_full_df_01 = df_con_fee[(df_con_fee['seller_nickname'].isin(seller_full)) & (df_con_fee['logistic_type'] == 'fulfillment')].copy()
seller_full_df_01 = df_con_fee[(df_con_fee['seller_nickname'].isin(seller_full)) & (df_con_fee['logistic_type'].isin(['fulfillment', 'xd_drop_off', 'cross_docking']))].copy()
# seller_full_df_01 = df_con_fee[df_con_fee['seller_nickname'].isin(seller_full)].copy()

# seller_full_df_01['stock_weight'] = round(seller_full_df_01['stock'] / seller_full_df_01.groupby(['seller_nickname', 'logistic_type'])['stock'].transform('sum'),6)
seller_full_df_01['stock_weight'] = round(seller_full_df_01['stock'] / seller_full_df_01.groupby(['seller_nickname'])['stock'].transform('sum'),6)

seller_full_df_01['weight_fee_per_sale'] = round(seller_full_df_01['stock_weight'] * seller_full_df_01['fee_per_unit'],6)
seller_full_df_01['weight_price'] = round(seller_full_df_01['stock_weight'] * seller_full_df_01['price'],6)


seller_full_df_01['monthly_stock_rotation'] = seller_full_df_01.apply(lambda row: definir_rotacion(row['stock'], row['logistic_type']), axis=1)
seller_full_df_01['monthly_tgmv'] = round(seller_full_df_01['monthly_stock_rotation'] * seller_full_df_01['fee_per_unit'],6)
seller_full_df_01

In [ ]:
seller_full_df_01[seller_full_df_01['seller_nickname'] == 'HAKELINDEHAKELINDE202209121728'].groupby(by=['seller_nickname', 'logistic_type']).agg({'stock': 'sum', 'stock_value': 'sum', 'stock_tgmv': 'sum', 'item_id': 'count'}).reset_index().sort_values(['stock_tgmv'], ascending=False)

In [ ]:
pd.options.display.float_format = '{:.6f}'.format

In [ ]:
# seller_full_df_01[seller_full_df_01['logistic_type'] == 'fulfillment'].groupby(by=['seller_nickname'])[['stock','stock_value', 'stock_tgmv']].sum().reset_index().sort_values(['stock_tgmv'], ascending=False)

# seller_full_df_02 = seller_full_df_01.groupby(by=['seller_nickname', 'logistic_type']).agg({'stock': 'sum', 'stock_value': 'sum', 'stock_tgmv': 'sum', 'item_id': 'count', 'weight_fee_per_sale': 'sum', 'weight_price': 'sum'}).sort_values(['weight_fee_per_sale'], ascending=False).reset_index().copy()
seller_full_df_02 = seller_full_df_01.groupby(by=['seller_nickname']).agg({
    'stock': 'sum', 'stock_value': 'sum', 'stock_tgmv': 'sum'
    , 'item_id': 'count', 'weight_fee_per_sale': 'sum'
    , 'weight_price': 'sum', 'monthly_stock_rotation': 'sum', 'monthly_tgmv': 'sum'
    # }).sort_values(['weight_fee_per_sale'], ascending=False).reset_index().copy()
    }).sort_values(['monthly_tgmv'], ascending=False).reset_index().copy()


seller_full_df_02['monthly_stock_rotation_share'] = round(seller_full_df_02['monthly_stock_rotation'] / seller_full_df_02['stock'],6)
seller_full_df_02['monthly_tgmv_share'] = round(seller_full_df_02['monthly_tgmv'] / seller_full_df_02['monthly_tgmv'].sum(),6)
seller_full_df_02['monthly_tgmv_share_running_sum'] = seller_full_df_02['monthly_tgmv_share'].cumsum()

seller_full_df_02

In [ ]:
# relevantes para el negocio
top_sellers = seller_full_df_02[seller_full_df_02['monthly_tgmv_share_running_sum'] <= 0.7].copy()
top_sellers

In [ ]:
#@title boxplot de precios
import matplotlib.pyplot as plt

def bx1(col_name):

    # Create a boxplot
    plt.boxplot(top_sellers[col_name], vert=False)

    # Add title and labels
    plt.title(col_name)
    # plt.xlabel('Data')
    # plt.ylabel('Values')

    # Show the plot
    plt.show()

def bx2(col_name):

    plt.boxplot(top_sellers[col_name], vert=False, showfliers=False)

    # Add title and labels
    plt.title(col_name)
    # plt.xlabel('Data')
    # plt.ylabel('Values')

    # Show the plot
    plt.show()

In [ ]:
# porcentaje de rotacion de el inventario de forma mensual
bx1('monthly_stock_rotation_share') # con outliers
bx2('monthly_stock_rotation_share') # sin outliers
top_sellers['monthly_stock_rotation_share'].describe()

In [ ]:
# weight_fee_per_sale	weight_price	monthly_stock_rotation
# porcentaje de rotacion de el inventario de forma mensual
bx1('weight_fee_per_sale') # con outliers
bx2('weight_fee_per_sale') # sin outliers
top_sellers['weight_fee_per_sale'].describe()

In [ ]:
# weight_fee_per_sale	weight_price	monthly_stock_rotation
# porcentaje de rotacion de el inventario de forma mensual
bx1('monthly_stock_rotation') # con outliers
bx2('monthly_stock_rotation') # sin outliers
top_sellers['monthly_stock_rotation'].describe()

In [ ]:
# Precio promedio ponderado de los productos
bx1('weight_price') # con outliers
bx2('weight_price') # sin outliers
top_sellers['weight_price'].describe()

In [ ]:
# Cantidad de publicaciones por vendedor
bx1('item_id') # con outliers
bx2('item_id') # sin outliers
top_sellers['item_id'].describe()

In [ ]:
seller_full_df_02.head(10)

In [ ]:
seller_full_df_02.tail(10)

Mi definicion de relevante para el negocio son todos aquellos vendedores que aporten la mayor utilidad al negocio de forma mensual

Aunque existe metricas como:

*   Mercado Lider
*   Reputacion
*   Stock virtual
*   Programa de sellers

Considero que un buen perfil es aquel que comparte caracteristicas con los top sellers

Algunas de las caracteristicas clave desde mi punto de vista son:
*   Precio promedio por venta
*   Porcentaje mensual de rotacion del inventario
*   Cantidad de productos diferentes en venta
*   Comision promedio por venta

In [ ]:
# RELEVANTES PARA EL NEGOCIO
# sellet_logistic

# BUEN PERFIL

# QUE ESTRATEGIA UTILIZARIAS PARA IDENTIFICAR ESTAS CARACTERISTICAS?

# # hipotesis

#     # (1) vendedores con stock en el almacen (50% o mas)
#     # porque hay vendedores que no tienen todo su stock en el almacen?
#     # quienes son los vendedores con mayor potencial de tgmv
#     # que vendedores tienen alto potencial y son XD
#     # vendedores que son los unicos con ciertas marcas de inventario